In [26]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio import SeqIO 


# Import the sequence(s)

In [50]:
seq_record = SeqIO.read('ELF3.fa','fasta')
#seq_record.id 
seq_record.seq 

Seq('MKRGKDEEKILEPMFPRLHVNDADKGGPRAPPRNKMALYEQLSIPSQRFGDHGT...SKP', SingleLetterAlphabet())

# Call blast from the NCBI website

In [51]:
result_handle = NCBIWWW.qblast("blastp", "nr", seq_record.seq) 

# Save and open the results

In [52]:
with open('results.xml', 'w') as save_file: 
    blast_results = result_handle.read() 
    save_file.write(blast_results)

E_VALUE_THRESH = 1e-20 
for record in NCBIXML.parse(open("results.xml")): 
     if record.alignments: 
        print("\n") 
        print("query: %s" % record.query[:100]) 
        for align in record.alignments: 
            for hsp in align.hsps: 
                if hsp.expect < E_VALUE_THRESH: 
                    print("match: %s " % align.title[:100])
                    



query: unnamed protein product
match: ref|NP_180164.1| hydroxyproline-rich glycoprotein family protein [Arabidopsis thaliana] >sp|O82804.1 
match: emb|VYS53527.1| unnamed protein product [Arabidopsis thaliana] 
match: gb|KAG7642078.1| hypothetical protein ISN44_As02g020390 [Arabidopsis suecica] 
match: gb|KAG7637467.1| hypothetical protein ISN45_At02g019970 [Arabidopsis thaliana x Arabidopsis arenosa] 
match: emb|CAA0371864.1| unnamed protein product [Arabidopsis thaliana] 
match: emb|CAA72719.1| nematode responsive protein [Arabidopsis thaliana] 
match: emb|CAD5319528.1| unnamed protein product [Arabidopsis thaliana] 
match: gb|OAP09466.1| PYK20 [Arabidopsis thaliana] 
match: gb|EFH55167.1| hypothetical protein ARALYDRAFT_481449 [Arabidopsis lyrata subsp. lyrata] 
match: emb|CAE6010521.1| unnamed protein product [Arabidopsis arenosa] 
match: gb|KAG7572294.1| hypothetical protein ISN44_As09g006620 [Arabidopsis suecica] 
match: ref|XP_020883196.1| protein EARLY FLOWERING 3 isoform X2 